In [1]:
import numpy as np

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

import tensorwatch as tw

import tensorpack.dataflow as df

import tqdm


In [2]:
# prepare dataset
ds = df.dataset.Mnist('train')
augmentors_variation = [
    df.imgaug.Resize((28, 28)),
    df.imgaug.CenterPaste((32, 32)),
    df.imgaug.RandomCrop((28, 28)),

    #df.imgaug.MapImage(lambda v: v.reshape(784))
    df.imgaug.MapImage(lambda v: np.expand_dims(v, 0))
]
ds = df.AugmentImageComponent(ds, augmentors_variation)
ds = df.PrefetchData(ds, nr_prefetch=100, nr_proc=4)
ds = df.PrintData(ds)
ds = df.BatchData(ds, batch_size=100, remainder=False, use_list=False)
ds = df.PrintData(ds)

[0608 17:33:04 @fs.py:101] WRN Env var $TENSORPACK_DATASET not set, using /Users/tmquan/tensorpack_data for datasets.
[0608 17:33:04 @parallel.py:194] [MultiProcessPrefetchData] Will fork a dataflow more than one times. This assumes the datapoints are i.i.d.
[0608 17:33:04 @argtools.py:146] WRN Starting a process with 'fork' method is not safe and may consume unnecessary extra memory. Use 'forkserver' method (available after Py3.4) instead if you run into any issues. See https://docs.python.org/3/library/multiprocessing.html#contexts-and-start-methods


In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=-1)


In [ ]:
model = Net()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    
# train
step = 0
for epoch in range(30):
    for minibatch in ds.get_data():
        images = Variable(torch.from_numpy(minibatch[0]))
        labels = Variable(torch.from_numpy(minibatch[1].astype(np.int64)))

        optimizer.zero_grad()
        output = model(images)
        loss = torch.nn.functional.nll_loss(output, labels)
        loss.backward()
        optimizer.step()

        l = loss.data #[0]

        pred = output.data.max(1, keepdim=True)[1]
        a = pred.eq(labels.data.view_as(pred)).cpu().sum() #/ 128.0
        step += 1

        if step % 100 == 0:
            print('epoch:{:02d}, step:{:06d}, loss:{:.3f}, accuracy:{:.3f}'.format(epoch, step, l, a))

epoch:00, step:000100, loss:2.289, accuracy:15.000
epoch:00, step:000200, loss:2.299, accuracy:15.000
epoch:00, step:000300, loss:2.290, accuracy:16.000
epoch:00, step:000400, loss:2.266, accuracy:17.000
epoch:00, step:000500, loss:2.259, accuracy:21.000
epoch:00, step:000600, loss:2.234, accuracy:26.000
epoch:01, step:000700, loss:2.135, accuracy:26.000
epoch:01, step:000800, loss:2.073, accuracy:30.000
epoch:01, step:000900, loss:1.702, accuracy:51.000
epoch:01, step:001000, loss:1.609, accuracy:46.000
epoch:01, step:001100, loss:1.502, accuracy:45.000
epoch:01, step:001200, loss:1.532, accuracy:48.000
epoch:02, step:001300, loss:1.342, accuracy:57.000
epoch:02, step:001400, loss:1.200, accuracy:61.000
epoch:02, step:001500, loss:1.284, accuracy:56.000
epoch:02, step:001600, loss:1.116, accuracy:64.000
epoch:02, step:001700, loss:1.058, accuracy:61.000
epoch:02, step:001800, loss:1.049, accuracy:67.000
epoch:03, step:001900, loss:1.204, accuracy:57.000
epoch:03, step:002000, loss:1.1